In [1]:
from opentrons import robot, containers, instruments
import pandas as pd
import argparse
import sys

# Configuration
SOURCE_SLOTS = ['D1','D2','D3','B2']

## Load our files
#parser = argparse.ArgumentParser(description="Run a DNA build on an Opentrons OT-1 robot.")
#parser.add_argument('-l', '--layout', required=False, help="A CSV file describing the layout of the sourcep plates.")
#parser.add_argument('-b', '--build-plan', required=True, help="A CSV file describing the build plan.")
#parser.add_argument('-s', '--simulate', required=False, action="store_true", help="Simulate the robot run and print command output.")
#args = parser.parse_args()

#plan = pd.read_csv(args.build_plan, usecols=['Gene','Wells'])
plan = pd.read_csv('../synth1/buildable-fragments_39-50.csv', usecols=['Gene','Wells'])



if 1 == 0: #args.layout:
    # We were given an explicit layout
    layout = pd.read_csv(args.layout)
    layout = layout.set_index('Position').to_dict()['Name'] # Turn into a location->name dict
else:
    plate_names = plan['Wells'].str.split('-').str[0].unique()

    if len(plate_names) > len(SOURCE_SLOTS):
        print("Error: This build plan requires too many source plates.")
        sys.exit(1)

    layout = list(zip(SOURCE_SLOTS[:len(plate_names)], plate_names))

    slots = pd.Series(SOURCE_SLOTS)
    columns = sorted(slots.str[0].unique())
    rows = sorted(slots.str[1].unique(), reverse=True)

    layout_table = pd.DataFrame(index=rows, columns=columns)
    layout_table.fillna("", inplace=True)

    for slot, plate in layout:
        layout_table.loc[slot[1], slot[0]] = plate

    print("Please arrange the plates in the following configuration:")
    print()
    print(layout_table)
    print()
    input("Press enter to continue")

    layout = dict(layout)


Please arrange the plates in the following configuration:

                    B                   D
3                      pSHPs0826B426849MU
2  pSHPs0826B426850MU  pSHPs0807B412040MU
1                      pSHPs0807B412039MU

Press enter to continue


In [2]:
# Configure the robot

#  Layout:
#    A     B       C      D      E
#  3 p200  master  master source p10
#  2       dest    dest   source p10
#  1       trash          source p10
#

if 1 == 1: #args.simulate:
    print("Simulating protcol run")
    robot.connect()
else:
    port = robot.get_serial_ports_list()[0]
    print("Connecting robot to port {}".format(port))
    robot.connect(port)

robot.home()

p200_tipracks = [
    containers.load('tiprack-200ul', 'A3'),
]

p10_tipracks = [
    containers.load('tiprack-10ul', 'E2'),
]

p10s_tipracks = [
    containers.load('tiprack-10ul', 'E3'),
    containers.load('tiprack-10ul', 'E1'),
]

trash = containers.load('point', 'B1', 'holywastedplasticbatman')
master = containers.load('PCR-strip-tall', 'C3')

dest_plates = [
    containers.load('96-PCR-tall', 'C2'),
    containers.load('96-PCR-tall', 'B2')
]

source_plates = {}
for slot, plate in layout.items():
    source_plates[plate] = containers.load('96-flat', slot)

p10 = instruments.Pipette(
    axis='a',
    max_volume=10,
    min_volume=0.5,
    tip_racks=p10_tipracks,
    trash_container=trash,
    channels=8,
    name='p10-8'
)

p10s = instruments.Pipette(
    axis='a',
    max_volume=10,
    min_volume=0.5,
    tip_racks=p10s_tipracks,
    trash_container=trash,
    channels=1,
    name='p10-8s'
)

p200 = instruments.Pipette(
    axis='b',
    max_volume=200,
    min_volume=20,
    tip_racks=p200_tipracks,
    trash_container=trash,
    channels=1,
    name='p200-1'
)


Simulating protcol run


In [3]:
# Run the protocol

# Load dest plates

# Distribute down the plates

max_reactions = 96

if len(plan) <= max_reactions:
    num_reactions = len(plan)
    print("Will run {} reactions".format(num_reactions))
else:
    num_reactions = max_reactions
    print("Too many reactions, will only run first {} reactions".format(num_reactions))

#num_reactions = len(plan)
num_rows = num_reactions // 8
print(num_rows)
all_wells = dest_plates[0].wells() + dest_plates[1].wells()

print("Building {} reactions in {} rows".format(num_reactions, num_rows))

p10.pick_up_tip()

for i in range(num_rows):
    p = i // 12
    r = i % 12

    print("Transferring master mix to plate {} row {}".format(p, r))
    p10.transfer(8, master['A1'], dest_plates[p].rows(r).bottom(), blow_out=True, touch_tip=True, new_tip='never')

p10.drop_tip()

j = 0

# Add multiples of mastermix to plates with multiple fragments
p10s.pick_up_tip()
for i, construct in plan.iterrows():
    vol = 8 * (len(construct['Wells'].split(',')) - 1)
    j = j + (len(construct['Wells'].split(',')) - 1)
    if vol > 0:
        print("Adding {} ul to well {} for multifragment assembly".format(vol, i))
        p10s.transfer(vol, master['A1'], all_wells[int(i)].bottom(), blow_out=True, touch_tip=True, new_tip='never')
    if i == num_reactions:
        break
p10s.drop_tip()

master_reactions = (num_reactions + j) + 8
print("Make {} rxns of master mix".format(master_reactions))

master_mix = pd.DataFrame({
    'Component':['Cutsmart','ATP','Vector','T4 Ligase','BbsI','H2O','Total'],
    'Amount':[master_reactions,master_reactions,(0.25*master_reactions),(master_reactions*(50/96)),(master_reactions*(6/96)),(5.166*master_reactions),(master_reactions*8)]
})
master_mix
## Move source DNA into dest mastermixes
#for i,construct in plan.iterrows():
#    print("Building gene {} {}".format(i, construct['Gene']))
#    fragments = construct['Wells'].split(',')
#    for fragment in fragments:
#        plate, well = fragment.split('-')
#        print("    Adding fragment from plate {} well {}".format(plate,well))
#        p10s.transfer(2, source_plates[plate].wells(well).bottom(), all_wells[int(i)].bottom(), blow_out=True, touch_tip=True, mix_before=(3,5))
#    if i == num_reactions:
#        break
#if args.simulate:
#    print()
#    print("Ran commands:")
#    for c in robot.commands():
#        print(c)
#

Too many reactions, will only run first 96 reactions
12
Building 96 reactions in 12 rows
Transferring master mix to plate 0 row 0
Transferring master mix to plate 0 row 1
Transferring master mix to plate 0 row 2
Transferring master mix to plate 0 row 3
Transferring master mix to plate 0 row 4
Transferring master mix to plate 0 row 5
Transferring master mix to plate 0 row 6
Transferring master mix to plate 0 row 7
Transferring master mix to plate 0 row 8
Transferring master mix to plate 0 row 9
Transferring master mix to plate 0 row 10
Transferring master mix to plate 0 row 11
Adding 8 ul to well 15 for multifragment assembly
Adding 8 ul to well 18 for multifragment assembly
Adding 8 ul to well 19 for multifragment assembly
Adding 8 ul to well 20 for multifragment assembly
Adding 8 ul to well 35 for multifragment assembly
Adding 8 ul to well 43 for multifragment assembly
Adding 8 ul to well 48 for multifragment assembly
Adding 8 ul to well 58 for multifragment assembly
Adding 8 ul to we

,Amount,Component
0,116.000000,Cutsmart
1,116.000000,ATP
2,29.000000,Vector
3,60.416667,T4 Ligase
4,7.250000,BbsI
5,599.256000,H2O
6,928.000000,Total


In [45]:
plan[:max_reactions].to_csv('../synth1/round1-assemblies.csv')

In [219]:
## FOR GIT REPOSITORY -- Queries the database for build ready constructs and then sets up the plates

from opentrons import robot, containers, instruments
import argparse
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
import os
import glob
import re
import math

## Take in required information

# Load files
#parser = argparse.ArgumentParser(description="Resuspend a plate of DNA on an Opentrons OT-1 robot.")
#parser.add_argument('-r', '--run', required=False, action="store_true", help="Send commands to the robot and print command output.")
#args = parser.parse_args()

letter = ["A","B","C","D","E","F","G","H"]
number = ["1","2","3","4","5","6","7","8","9","10","11","12"]

target_well = []
temp_well = 0

for n in number:
    for l in letter:
        temp_well = l + n
        target_well.append(temp_well)

targets = []

counter = 0
frag_num = 0

unique_plates = 0
number_plates = 3
#plates = ["pSHPs0807B412037MU", "pSHPs0807B412038MU"]
plates = []

gene_list = []
gene_num = 0

# Query the database and iterate through each json file
for file in glob.glob("../data/BBF10K*/*.json"):
    print(file)
    
    # Open and store the data within the json file
    with open(file,"r") as json_file:
        data = json.load(json_file)
    
    # Determine if it is build ready
    if data["status"]["build_ready"] != "TRUE":
        continue
    print("build ready")
    
    # Pull general information about the gene
    gene = data["id"]
    if gene_num == 10:
        break
    locations = data["location"]["fragments"]
    frag_num = len(locations)
    if frag_num > 2:
        continue
    print("number of fragments: ", frag_num)
    
    # Iterate through all of the fragments
    for fragment in locations:
        print(fragment)
        
        # Pulls out the location for a specific fragment
        frag_loc = data["location"]["fragments"][fragment]
        
        # Separate the well from the plate and determine how many unique plates there are
        plate_loc, well = frag_loc.split("_")
        plates.append(plate_loc)
        plates_pd = pd.Series(plates)
        unique_plates = len(plates_pd.unique())
        print("number on unique plates:", unique_plates)
        
        # If there are too many unique plates it roles back the counter and series of plates and moves to the next file
        if unique_plates > number_plates:
            
            plates = plates[:-1]
            print("too many unique plates")
            continue
        else:
            #row = [gene, plate_loc, well]
            #targets.append(row)
            gene_list.append(gene)
            gene_list_pd = pd.Series(gene_list)
            gene_num = len(gene_list_pd.unique())
            dest_well = target_well[gene_num-1]
            row = [gene, plate_loc, well, dest_well]
            targets.append(row)
            print("gene_num", gene_num)
            
            if gene_num == 10:
                break
        print()
    #counter = counter + 1

print("unique plates", plates_pd.unique())
        
targets = np.array(targets)
plan = pd.DataFrame({
    "Plate" : targets[:,1],
     "Gene" : targets[:,0],
     "Well" : targets[:,2],
     "Destination" : targets[:,3]
    })
plan = plan[["Gene","Plate","Well","Destination"]]
plan
        


../data/BBF10K_000001/BBF10K_000001.json
build ready
number of fragments:  1
BBF10K_000001_1
number on unique plates: 1
gene_num 1

../data/BBF10K_000002/BBF10K_000002.json
build ready
number of fragments:  1
BBF10K_000002_1
number on unique plates: 2
gene_num 2

../data/BBF10K_000003/BBF10K_000003.json
build ready
number of fragments:  1
BBF10K_000003_1
number on unique plates: 3
gene_num 3

../data/BBF10K_000004/BBF10K_000004.json
build ready
number of fragments:  1
BBF10K_000004_1
number on unique plates: 3
gene_num 4

../data/BBF10K_000005/BBF10K_000005.json
build ready
number of fragments:  1
BBF10K_000005_1
number on unique plates: 4
too many unique plates
../data/BBF10K_000006/BBF10K_000006.json
build ready
number of fragments:  2
BBF10K_000006_1
number on unique plates: 3
gene_num 5

BBF10K_000006_2
number on unique plates: 3
gene_num 5

../data/BBF10K_000007/BBF10K_000007.json
build ready
number of fragments:  2
BBF10K_000007_1
number on unique plates: 3
gene_num 6

BBF10K_000

,Gene,Plate,Well,Destination
0,BBF10K_000001,pSHPs0826B426849MU,B6,A1
1,BBF10K_000002,pSHPs0807B412037MU,A1,B1
2,BBF10K_000003,pSHPs0807B412038MU,F3,C1
3,BBF10K_000004,pSHPs0807B412037MU,A3,D1
4,BBF10K_000006,pSHPs0807B412037MU,F8,E1
5,BBF10K_000006,pSHPs0807B412037MU,G8,E1
6,BBF10K_000007,pSHPs0807B412037MU,E6,F1
7,BBF10K_000007,pSHPs0807B412037MU,F6,F1
8,BBF10K_000009,pSHPs0807B412038MU,D5,G1
9,BBF10K_000009,pSHPs0826B426849MU,B5,G1


In [101]:
array = []
plate = "first"
array.append(plate)
print(array)
array.append("another")
array.append("and")
array.append("last")

print(array[:-1])


['first']
['first', 'another', 'and']


In [175]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
import os
import glob
import re
import math

import shutil

counter = 0

# Imports the csv containing all of our data
data = pd.read_csv("./testing/data_testing/10K_CDS.csv")

# Takes in the 
for file in glob.glob("./testing/json/template.json"):
        print(file)
        with open(file,"r") as template_json:
            template = json.load(template_json)
            #print(template)

# Iterates through all of the items in the spreadsheet
for index, row in data.iterrows():
    
    # Forces it to only run a certain number of times
    counter = counter + 1
    if counter == 11:
        break

    # Generate the ID#
    gene = row['gene_name']
    seq = row['sequence']
    number = str(index + 1).zfill(6)
    idnum = "BBF10K_" + number
    
    #State the path to house the new set of directories
    path = "../old_files/test_db_upload/{}".format(idnum)
    
    # Only create a new directory if there are no existing directories
    if os.path.exists(path):
        print("Directory {} already exists".format(idnum))
    else:
        # Generates a new directory with the ID# as its name
        os.makedirs(path)
        
        # Generate the fasta file with the gene sequence
        fasta = open("./{}/{}.fasta".format(path,idnum),"w+")
        fasta.write(">{}".format(gene))
        fasta.write("\n")
        fasta.write(seq)
        fasta.close()
        
        # Fill in the template json file with the information from each row
        template["gene_id"] = idnum
        template["gene_name"] = gene
        template["sequence"]["original_sequence"] = seq
        template["sequence"]["optimized_sequence"] = seq
        template["author"]["name"] = row["author"]
        template["author"]["email"] = row["author_email"]
        template["author"]["affiliation"] = row["author_affiliation"]
        template["author"]["project"] = row["author_project"]
        template["info"]["type"]["cloning_method"] = row["cloning_method"]
        template["info"]["type"]["part_type"] = row["part_type"]
        template["info"]["type"]["build_type"] = row["build_type"]
        template["info"]["safety"] = row["safety"]
        template["info"]["collection"] = row["collection"]
        template["info"]["other_tags"] = row["other_tags"]
        
        
        
        
        with open("./{}/{}.json".format(path,idnum),"w+") as json_file:
            json.dump(template,json_file,indent=2)
    
    
    

./testing/json/template.json
{'gene_name': '', 'gene_id': '', 'authors': [{'name': '', 'email': '', 'affiliation': '', 'project': ''}], 'info': {'database_links': [''], 'type': {'cloning_method': '', 'part_type': '', 'build_type': ''}, 'safety': '', 'collection': '', 'other_tags': ''}, 'sequence': {'original_sequence': '', 'optimized_sequence': '', 'fragment_sequences': {}}, 'status': {'ordered': '', 'will_build': '', 'build_ready': '', 'building': '', 'build_complete': '', 'build_attempts': [{'build_well': '', 'build_date': '', 'build_number': '', 'build_outcome': ''}]}, 'location': {'fragments': {}}, 'cloned': [{'vector': '', 'organism': '', 'location': ['']}], 'dates': {'ordered': '', 'build_ready': '', 'complete': ''}}
BBF10K_000001
MMSYN1_0001
BBF10K_000002
MMSYN1_0002
BBF10K_000003
MMSYN1_0003
BBF10K_000004
MMSYN1_0004
BBF10K_000005
MMSYN1_0005
BBF10K_000006
MMSYN1_0006
BBF10K_000007
MMSYN1_0007
BBF10K_000008
MMSYN1_0008
BBF10K_000009
MMSYN1_0009
BBF10K_000010
MMSYN1_0010


In [215]:
letter = ["A","B","C","D","E","F","G","H"]
number = ["1","2","3","4","5","6","7","8","9","10","11","12"]

target_well = []
temp_well = 0

for n in number:
    for l in letter:
        temp_well = l + n
        target_well.append(temp_well)
        


D1
